<a href="https://colab.research.google.com/github/machineslearningstuff/DeepChatBot/blob/master/bert_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import os
import sys

print(sys.argv)

print('Hello world!')

['/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py', '-f', '/root/.local/share/jupyter/runtime/kernel-5f8efbe5-c49f-49ed-b2bc-582f028634f7.json']
Hello world!
